# install required librares

In [2]:
! pip install numpy pandas faiss-cpu requests 

  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/4c/e1/657eb537027b2d7aa0f0ccfc58aee6fe0252ea3d9e49472aecc5c7f30992/faiss_cpu-1.8.0.post1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB 640.0 kB/s eta 0:00:23
   ---------------------------------------- 0.1/14.6 MB 656.4 kB/s eta 0:00:23
   ---------------------------------------- 0.1/14.6 MB 744.7 kB/s eta 0:00:20
   ---------------------------------------- 0.1/14.6 MB 744.7 kB/s eta 0:00:20
   ---------------------------------------- 0.1/14.6 MB 744.7 kB/s eta 0:00:20
   ---------------------------------------- 0.1/14.6 MB 500.5 kB/s eta 0:00:29
   ---------------------------------------- 0.1/14.6 MB 500.5 kB/s eta 0:00:29
    --------------------------------------- 0.3/14.6 MB 714.4 kB/s eta 0:00:21
    --------------------------------------- 0.3/14.6 MB 680.9 kB/s eta 0:00:22
 

In [1]:
import pandas as pd

# read the data 

In [2]:
df=pd.read_csv(r'D:\abdo\AI\projects\recommender\model\LLMs\movies_with_tags_rating.csv')

In [3]:
df.head()

,index,title,film_rate,overview,cover_url,cast,reviews,imdbId,director,genre,rating,tag
0,0,Before and After,6.1,['Two parents deal with the effects when their...,https://m.media-amazon.com/images/M/MV5BOWJmOD...,"Meryl Streep, Liam Neeson, Edward Furlong",While not a terrible movie it has a few seriou...,115645,Barbet Schroeder,"Crime, Drama, Mystery",3.05,NaN
1,1,Anne Frank Remembered,8.1,"[""Tells the story of the Frank family and pain...",https://m.media-amazon.com/images/M/MV5BNGZjNz...,"Isa Baschwitz, Mary Bos, Kenneth Branagh",The Diary of Anne Frank is the second best-sel...,112373,Jon Blair,"Documentary, Biography, War",4.05,NaN
2,2,The Young Poisoner's Handbook,7.0,"[""This film is based on a true story about a B...",https://m.media-amazon.com/images/M/MV5BMTg0MT...,"Tobias Arnold, Ruth Sheen, Roger Lloyd Pack",Director Benjamin Ross has done a terrific job...,115033,Benjamin Ross,"Comedy, Crime, Drama",3.50,NaN
3,3,If Lucy Fell,4.9,"[""Two NYC roommates have a pact to jump off Br...",https://m.media-amazon.com/images/M/MV5BMmFjYW...,"Sarah Jessica Parker, Eric Schaeffer, Ben Stiller","...to Eric Schaeffer, but he's not the leading...",116606,Eric Schaeffer,"Comedy, Romance",4.00,NaN
4,4,Steal Big Steal Little,4.7,"[""Ruben and Robby are twin brothers, adopted b...",https://m.media-amazon.com/images/M/MV5BNjVhMz...,"Andy Garcia, Alan Arkin, Rachel Ticotin",During the time this was being shot I was spen...,114536,Andrew Davis,Comedy,2.35,NaN


# represent the data

In [17]:
def representer(row):
    representation=f"""
      title: {row['title']}
      imdbId: {row['imdbId']}
      director: {row['director']}
      cast: {row['cast']}  
      tag: {row['tag']}  
      genre: {row['genre']} 
      overview: {row['overview']}
      reviews: {row['reviews']}
    """
    return representation

In [18]:
df['representation']= df.apply(representer, axis=1)

In [19]:
print(df['representation'].values[0])


      title: Before and After
      imdbId: 115645
      director: Barbet Schroeder
      cast: Meryl Streep, Liam Neeson, Edward Furlong  
      tag: nan  
      genre: Crime, Drama, Mystery 
      overview: ['Two parents deal with the effects when their son is accused of murdering his girlfriend.', "When teenage son Jacob is being accused of murdering his girlfriend, the well-respected and close-knit Ryan family is in turmoil. Jacob flees, father Ben destroys possible evidence, the village community turns hostile and mother Carolyn is forced to temporarily close her doctor's practice. Then Jacob gets arrested and soon finds himself and his family entangled in a web of truth, trust and lies, all on his way to court.âSwie Tio <sweet_t_prod@hotmail.com>"]
      reviews: While not a terrible movie it has a few serious problems. The problems start with the casting. Edward Furlong is cast to play the 16 year old son of parents played by Meryl Streep and Liam Neeson, but at age 16 he app

# embedding

In [10]:
import faiss
import numpy as np
import requests

In [20]:
dim = 4096 # dimension of the vectors retured by llama 2 
index=faiss.IndexFlatL2(dim)
x=np.zeros((len(df['representation']),dim),dtype='float32') #llama 2 embedding

In [21]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# embedd the data set features

In [22]:
for i, representation in enumerate(df['representation']):
    if i % 100 == 0:
        print('embedding movies done =' , i) # just to see the progress
    response = requests.post('http://localhost:11434/api/embeddings', 
                             json={
                                    'model': 'llama2',
                                    'prompt': representation
                             })
    embedding = response.json().get('embedding', [])
    if embedding:
        x[i] = np.array(embedding, dtype='float32')
index.add(x) # add the vectors to the index 
# instead of making a vector and then fill the missing values with padding values, we can make a vector of zeros and fill the values that are not zeros


embedding movies done = 0
embedding movies done = 100
embedding movies done = 200
embedding movies done = 300
embedding movies done = 400
embedding movies done = 500
embedding movies done = 600
embedding movies done = 700
embedding movies done = 800
embedding movies done = 900
embedding movies done = 1000
embedding movies done = 1100
embedding movies done = 1200
embedding movies done = 1300
embedding movies done = 1400
embedding movies done = 1500
embedding movies done = 1600
embedding movies done = 1700
embedding movies done = 1800
embedding movies done = 1900
embedding movies done = 2000
embedding movies done = 2100
embedding movies done = 2200
embedding movies done = 2300
embedding movies done = 2400
embedding movies done = 2500
embedding movies done = 2600
embedding movies done = 2700
embedding movies done = 2800
embedding movies done = 2900
embedding movies done = 3000
embedding movies done = 3100
embedding movies done = 3200
embedding movies done = 3300
embedding movies done = 34

## save the index file

In [23]:
faiss.write_index(index,'index') # save the index to your local pc to save time the next time we run the code

## if u have an index file already 

In [39]:
index=faiss.read_index('index') # load the index file from your pc 
                                                        #**make sure that index file is in the same directory as the code

# manual recommendation

In [50]:
df[df.title.str.contains('inception', na=False , case=False)]

,index,title,film_rate,overview,cover_url,cast,reviews,imdbId,director,genre,rating,tag,representation
8268,8268,Inception,8.8,['A thief who steals corporate secrets through...,https://m.media-amazon.com/images/M/MV5BMjAxMz...,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...","My 3rd time watching this movie! Yet, it still...",1375666,Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",4.4,NaN,\n title: Inception\n imdbId: 137566...


In [51]:
favourite_movie_data=df.iloc[8268]

In [54]:
favourite_movie_data['representation']

'\n      title: Inception\n      imdbId: 1375666\n      director: Christopher Nolan\n      cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page  \n      tag: nan  \n      genre: Action, Adventure, Sci-Fi, Thriller \n      overview: [\'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.\', "Dom Cobb is a skilled thief, the absolute best in the dangerous art of extraction, stealing valuable secrets from deep within the subconscious during the dream state, when the mind is at its most vulnerable. Cobb\'s rare ability has made him a coveted player in this treacherous new world of corporate espionage, but it has also made him an international fugitive and cost him everything he has ever loved. Now Cobb is being offered a chance at redemption. One last job could give him his life back but only if he can accomplish the i

In [33]:
def recommend(movie):
    # Step 1: Search for the given favourite movie
    search_result = df[df['title'].str.contains(movie, case=False, na=False)]  # case insensitive search
    
    if search_result.empty:
        print(f"No movie found for '{movie}' in the dataset.")
        return

    favourite_movie_data = search_result.iloc[0]  # Take the first match from search results

    # Step 2: Request embeddings from the API
    response = requests.post('http://localhost:11434/api/embeddings', # after downloading llama2 from ollama
                         json={
                             'model': 'llama2', 
                             'prompt': favourite_movie_data['representation']
                         })

  
    if response.status_code != 200:
        print(f"Error in getting embedding: {response.status_code}")
        return
    
    embedding = response.json().get('embedding', [])
    
    if not embedding:
        print("No embedding returned.")
        return
    
    # Step 3: Search the nearest embeddings in the index
    D, Ids = index.search(np.array(embedding, dtype='float32').reshape(1, -1), 5)  # Get top 5 similar movies

  
    recommended_movies_ids = Ids.flatten()  
    recommended_movies = df.iloc[recommended_movies_ids]  # Retrieve movie data

    print("Next movies to watch based on your choice:")
    for _, row in recommended_movies.iterrows():
        print(f"- {row['title']} | Genre: {row['genre']}")


In [34]:
test=input('what is your favourite movie?')
recommend(test)

AssertionError: 